<img src="../data/logo.png" width="420px">

# <span style="font-family:Cascadia mono; color:#db5a5a">TODOIST AUTOMATION - DIEGO</span>

## <span style="font-family:Cascadia mono; color:#db6a6a">Settings</span>

### Libraries

In [ ]:
from todoist_api_python.api import TodoistAPI
from datetime import datetime, timedelta, timezone
import functions as fun
import random
import json
import time
import os
from dotenv import load_dotenv

load_dotenv()

### Message

In [ ]:
today = datetime.today()

In [ ]:
messages = [f'Todoist Automation for {today.strftime("%Y-%m-%d")}']

### Runtime

In [ ]:
start_time = time.time()

### API

In [ ]:
api_token = os.getenv("API_TOKEN_DIEGO")
api = TodoistAPI(api_token)

## <span style="font-family:Cascadia mono; color:#db6a6a">Data</span>

#### Projects

In [ ]:
all_projects = fun.getProjects()
projects_dict_id, projects_dict_name = fun.getProjectsDicts(all_projects)

#### Tasks

In [ ]:
def refreshTasks():
    all_tasks = fun.getTasks()
    task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
    return all_tasks, task_dict_id, task_dict_name

In [ ]:
all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### Sections

In [ ]:
all_sections = fun.getSections()
sections_dict_id, sections_dict_name = fun.getSectionsDicts(all_sections)

#### Test

In [ ]:
index = random.randint(0,len(all_tasks))
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']])
except TypeError:
    print(all_tasks[index]['content'], "-", "No date" , "-",projects_dict_id[all_tasks[index]['project_id']],)

#### Labels

In [ ]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

#### F1 Calendar

In [ ]:
def getF1Calendar():
    return True


## <span style="font-family:Cascadia mono; color:#db6a6a">Orders</span>

#### <span style="font-family:Cascadia mono; color:silver">Create and edit tasks</span>

In [ ]:
def createTask(
    content = "Tarea creada automáticamente",
    description = None,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
    order = 1,
    labels = [],
    priority = 1,
    comment_count = 0,
    due_string = None,
    due_date = None,
    due_datetime = None,
    due_lang = None,
    assignee_id = None,
    duration = None,
    duration_unit = None
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            content = content,
            description = description,
            due_string = due_string,
            due_date = due_date,
            due_datetime = due_datetime,
            due_lang = due_lang,
            duration = duration,
            labels = labels,
            order = order,
            priority = fun.priorityInversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
            duration_unit = duration_unit
        )
        message = f'Task "{task.content}" ({task.id}) was created correctly'
        print(message)
        return message
    except Exception as error:
        print (error)
        return error
        
def editTask(
    task_id,
    content = None,
    assignee_id = None,
    description = None,
    due_date = None,
    due_string = None,
    due_datetime = None,
    labels = None,
    priority = None,
):
    task = api.get_task(task_id)
    labels = task.labels if labels == None else labels
    content = task.content if content == None else content
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        api.update_task(
            task_id = task_id,
            assignee_id = assignee_id,
            content = content,
            description = description,
            labels = labels,
            priority = priority,
            due_date = due_date,
            due_string = due_string,
            due_datetime = due_datetime,
        )
        message = f'Task {task.content} updated correctly to {content}'
        print(message)
        return message
    except Exception as error:
        print(error)
        return error

#### <span style="font-family:Cascadia mono; color:silver">Add duration labels</span>

In [ ]:
def updateTaskDurationLabel(task_id, task_duration):
    duration_label = fun.getDurationLabel(task_duration)
    task_id = task_id[0]
    task_labels_without_duration = fun.getLabelsWithoutDuration(task_id)
    task_labels_without_duration.append(duration_label)
    return editTask(task_id=task_id,
             labels=task_labels_without_duration)

In [ ]:
to_update = False
messages.append('Tasks to add duration label...')
for task in all_tasks:
    if task['duration'] != None and task['due'] != None:
        if fun.getDurationLabel(task['duration']['amount']) not in task['labels']:
            to_update = True
            message = updateTaskDurationLabel([task['id']], task['duration']['amount'])
            messages.append("- " + message)

if not to_update:
    messages = messages[:-1]
    message = 'No tasks to add duration label'
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### <span style="font-family:Cascadia mono; color:silver">Capitalize titles</span>

In [ ]:
to_update = False
messages.append("Tasks to capitalize it's title...")    
for task in all_tasks:
    if task["content"][0].upper() != task["content"][0]:
        to_update = True
        task_id = task["id"]
        message = editTask(task_id = task_id,
                           content = task["content"][0].upper() + task["content"][1:])
        messages.append("- " + message)

if not to_update:
    messages = messages[:-1]
    message = "No tasks to capitalize it's title"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### <span style="font-family:Cascadia mono; color:silver">Uncomplete recurring tasks</span>

In [ ]:
with open('../data/recurringtasksdiego', 'r') as archivo_json:
    recurringtasks = json.load(archivo_json)

In [ ]:
to_update = False
messages.append('Recurring tasks to uncomplete...')
for task_id in recurringtasks.keys():
    project_id = recurringtasks[task_id]
    if projects_dict_id[project_id] != "Archivados" and projects_dict_id[project_id] != "Tickler":
        print(task_dict_id[task_id])
        if fun.getTask(task_id).is_completed:
            to_update = True
            print(task['id'])
            fun.uncompleteTask(task_id)
            message = editTask(task_id=task_id,
                    due_string="No date")
            messages.append("- " + message)
        
if not to_update:
    messages = messages[:-1]
    message = "No recurring tasks to uncomplete"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### <span style="font-family:Cascadia mono; color:silver">Add labels to birthday tasks</span>

In [ ]:
to_update = False
messages.append('Birthday tasks to add labels...')
for task in all_tasks:
    if task['project_id'] == projects_dict_name['Cumpleaños'] and task['labels'] != ['Phone','Short']:
        to_update = True
        message = editTask(task_id=task['id'],
                 labels=['Phone','Short'])
        messages.append("- " + message)
        
if not to_update:
    messages = messages[:-1]
    message = "No birthday tasks to add labels"
    print(message)
    messages.append(message)
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### <span style="font-family:Cascadia mono; color:silver">Reminder and completion of habit tracking tasks not being completed</span>

In [ ]:
# fun.completeTask('')

#### <span style="font-family:Cascadia mono; color:silver">Fantasy task due</span>

In [ ]:
evaluate = True
if fun.getTask('4632052423').is_completed == True:
    fun.uncompleteTask('4632052423')
    editTask('4632052423', due_string='every friday 20:00')
    message = 'Fantasy task moved back to fridays'
    print (message)
    messages.append(message)
    evaluate = False

In [ ]:
to_update = False
if evaluate:
    if today.weekday() in [0,5,6] and fun.getTask('4632052423').due.date != today.strftime('%Y-%m-%d'):
        for task in all_tasks:
            if task['section_id'] == '51988025' and task['parent_id'] == '6968182312':
                try:
                    fantasydate = datetime.strptime(fun.getTask('4632052423').due.date, '%Y-%m-%d')
                    matchday = datetime.strptime(task['due']['date'][:10], '%Y-%m-%d')
                    if fantasydate > matchday > fun.getNextMonday():
                        message = 'Fantasy task moved to Tuesday'
                        messages.append(message)
                        editTask('4632052423', due_string="Tuesday")
                        to_update = True
                        all_tasks, task_dict_id, task_dict_name = refreshTasks()
                        break
                except TypeError:
                    pass
    if not to_update:
        message = 'Fantasy not moved'
        print(message)
        messages.append(message)

#### <span style="font-family:Cascadia mono; color:silver">Tasks with similar names</span>

## <span style="font-family:Cascadia mono; color:#db6a6a">Results</span>

### Export all tasks and it's status

In [ ]:
tasks = api.get_tasks()
recurringtasks={}
for task in tasks:
    if 'Recurring' in task.labels:
        recurringtasks[task.id]=task.project_id
with open('../data/recurringtasksdiego', 'w') as file:
    json.dump(recurringtasks, file)

### Runtime

In [ ]:
end_time = time.time()

In [ ]:
runtime = f'Runtime: {round(end_time- start_time,3)} seconds'
messages.append(runtime)
print(runtime)

### <span style="font-family:Cascadia code; color:#7a3aca">Message</span>

In [ ]:
body = messages[0] + "\n\n"
for message in messages[1:]:
    body = body + "- " + message + "\n"
    print(message,"\n")

### <span style="font-family:Cascadia code; color:#caba3a">Email</span>

In [ ]:
fun.sendEmail('diegorodgar17@gmail.com', 'Daily Todoist', body)